In [1]:
import os



In [2]:
!pwd

/home/srak/Documents/pers/end-to-end-chest-cancer/research


In [3]:
os.chdir("../")
!pwd

/home/srak/Documents/pers/end-to-end-chest-cancer


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [5]:
from ChestCancerClassification.constants import *
from ChestCancerClassification.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=config.root_dir,
            base_model_path=config.base_model_path,
            updated_base_model_path=config.updated_base_model_path,
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate= self.params.BATCH_SIZE,
            params_include_top= self.params.INCLUDE_TOP,
            params_weights= self.params.WEIGHTS,
            params_classes= self.params.LEARNING_RATE,
        )
        
        return prepare_base_model_config

In [10]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

2024-08-04 10:48:04.954861: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-04 10:48:05.697195: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-04 10:48:05.703601: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-04 10:48:08.263244: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [22]:
import tensorflow as tf
from ChestCancerClassification import logger

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig) -> None:
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        
        self.save_model(path=self.config.base_model_path, model=self.model)
        logger.info(f"Base Model saved in {self.config.base_model_path}")
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
                
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        full_model.summary()
        logger.info("Full Model Compiled Succesfully. ")
        return full_model
              
    
    def update_full_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )  
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    
      
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(filepath=path)

In [23]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_full_model()
except Exception as e:
    raise e

[2024-08-04 11:48:49,140: INFO: common: yaml file: config/config.yaml loaded completed.]
[2024-08-04 11:48:49,143: INFO: common: yaml file: params.yaml loaded completed.]
[2024-08-04 11:48:49,145: INFO: common: Created dir at path: artifacts]
[2024-08-04 11:48:49,147: INFO: common: Created dir at path: artifacts/prepare_base_model]
[2024-08-04 11:48:49,647: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2024-08-04 11:48:49,973: INFO: 2856370530: Base Model saved in artifacts/prepare_base_model/base_model.h5]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
            